In [13]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from sklearn.metrics import classification_report, confusion_matrix

from sklearn.metrics import plot_confusion_matrix

from sklearn.model_selection import KFold, cross_val_score, cross_val_predict
from sklearn.model_selection import train_test_split
from keras.utils import np_utils
import numpy as np
import csv
from matplotlib import pyplot as plt
import cv2
import pandas as pd
import pickle
import os

values=['div','times','+','-','0','1','2','3','4','5','6','7','8','9']
train_folder = './images_no_copies/'

train_data = []
labels = []

print("Creating input data...")
for foldername in os.listdir(train_folder):
    for filename in os.listdir(train_folder + foldername):
        img = cv2.imread(train_folder + foldername + "/" + filename, cv2.IMREAD_GRAYSCALE)
        currLabel=values.index(foldername)
        resized_img = cv2.resize(img, (45,45))
        img_data = resized_img.flatten() / 255 # flatten to 784 and normalize values
        train_data.append(img_data)
        labels.append(currLabel)

train_data = np.asarray(train_data)
labels = np.asarray(labels)

x_train, x_test, y_train, y_test = train_test_split(train_data, labels, test_size = 0.2, random_state = 101)
x_train = x_train.reshape(x_train.shape[0], 45, 45,1)
x_test = x_test.reshape(x_test.shape[0], 45, 45,1)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
# normalize inputs from 0-255 to 0-1
x_train = x_train / 255
x_test = x_test / 255
initialsplit=x_train
# one hot encode outputs
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

model = Sequential()
model.add(Conv2D(32, (5, 5), input_shape=(45, 45, 1), activation='relu'))
model.add(MaxPooling2D())


model.add(Conv2D(64, kernel_size = (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(64, kernel_size = (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


# Fit the model
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=15, batch_size=40,shuffle=True)
# Final evaluation of the model
scores = model.evaluate(x_test, y_test, verbose=0)
print("CNN Error: %.2f%%" % (100-scores[1]*100))

Creating input data...
Train on 27780 samples, validate on 6945 samples
Epoch 1/15
27780/27780 [==============================] - 29s 1ms/step - loss: 2.2466 - accuracy: 0.1757 - val_loss: 2.2368 - val_accuracy: 0.1587
Epoch 2/15
27780/27780 [==============================] - 29s 1ms/step - loss: 2.2355 - accuracy: 0.1788 - val_loss: 2.2350 - val_accuracy: 0.1785
Epoch 3/15
27780/27780 [==============================] - 29s 1ms/step - loss: 1.7355 - accuracy: 0.3943 - val_loss: 1.0213 - val_accuracy: 0.6688
Epoch 4/15
27780/27780 [==============================] - 29s 1ms/step - loss: 0.7079 - accuracy: 0.7794 - val_loss: 0.3906 - val_accuracy: 0.8878
Epoch 5/15
27780/27780 [==============================] - 29s 1ms/step - loss: 0.3687 - accuracy: 0.8942 - val_loss: 0.2440 - val_accuracy: 0.9320
Epoch 6/15
27780/27780 [==============================] - 30s 1ms/step - loss: 0.2712 - accuracy: 0.9237 - val_loss: 0.1831 - val_accuracy: 0.9473
Epoch 7/15
27780/27780 [======================

In [ ]:
test_folder='./test_dir/'

for filename in os.listdir(test_folder):
    img = cv2.imread(test_folder+"/" + filename, cv2.IMREAD_GRAYSCALE)
    resized_img = cv2.resize(img, (45,45))
    img_data = resized_img.flatten() / 255
    img_data = img_data.reshape(1, 45, 45, 1)
    img_data = img_data.astype('float32')
    result=model.predict(img_data)
    result=values[np.argmax(result)]
    print("Should be "+filename)
    print("Actual "+result)

In [15]:
y_pred=[]
print(x_test.shape)
print(y_test.shape)
for each in x_test:
    each=each.reshape(1,45,45,1)
    Y_pred = model.predict(each)
    y_pred.append(np.argmax(Y_pred, axis=1))

rounded_labels=np.argmax(y_test,axis=1)

print('Confusion Matrix')
confusionMat = confusion_matrix(rounded_labels,y_pred)
print(confusionMat)

(6945, 45, 45, 1)
(6945, 14)
Confusion Matrix
[[  37    0    2    0    0    1    0    0    0    0    0    0    0    0]
 [   0  134    0    0    0    0    0    0    0    0    0    0    0    0]
 [   0    0 1085    2    2    8    0    0    1    1    1    2    0    0]
 [   0    0    1 1230    0    3    1    0    0    0    0    0    0    0]
 [   0    0    1    0  340    2    0    0    0    2    2    1    0    0]
 [   0    0    2    0    1 1228    4    1    2    1    0    1    0    0]
 [   0    1    0    0    7   12 1193    1    1    1    0    4    7    2]
 [   0    0    0    0    2    1    4  487    3    3    0    1    1    1]
 [   0    0    2    0    0    1    0    1  307    0    4    0    1    5]
 [   0    0    0    0    0    1    1    0    1  183    4    0    3    0]
 [   0    0    0    0    3    2    1    0    5    1  168    0    2    0]
 [   0    0    1    0    2    0    3    1    1    1    0  123    2    0]
 [   0    2    0    0    1    2    1    0    1    1    0    0  142    0]
 [   

In [ ]:
#model.save('model.h5')
#print("Model saved as model.h5")
filename = 'model.sav'
pickle.dump(model, open(filename, 'wb'))